# Clean up data, convert to dataframes, and save to csv

In [7]:
import os 
import pandas as pd
from IPython.display import display, clear_output
from ipywidgets import widgets

labels = ['LogLevel','TimeStamp','LogVisibility','LogSeverity','entryType','entrySubType','eventType','EventType dependent strings']
loadingBar = widgets.IntProgress(value=0, min=0, max=100, step=1, description='Loading: 0%', bar_style='info', orientation='horizontal')
display(loadingBar)
count=1

for log in os.listdir('data'):
    lines = [line.rstrip('\n') for line in open('data/'+log)]
    info_lines = [[line[0:3],line[6:14]] +line[17:].split('|') for line in lines]
    uniform_lines = []
    for line in info_lines:
        new_line = line[:8]
        if len(line) > 8:
            new_line[7] = line[-(len(line)-7):]
        uniform_lines.append(new_line)
    df = pd.DataFrame.from_records(uniform_lines, columns=labels)
    df.to_csv('processed_data/'+log[-14:-4]+'.csv', sep=',', encoding='utf-8', index=False)
    loadingBar.value = int((count/len(os.listdir('data'))) * 100)
    loadingBar.description = 'Loading: ' + str(loadingBar.value) + '%'
    count+=1

A Jupyter Widget

In [8]:
loaded_df = pd.read_csv('processed_data/2017-05-21.csv')

In [9]:
loaded_df

,LogLevel,TimeStamp,LogVisibility,LogSeverity,entryType,entrySubType,eventType,EventType dependent strings
0,L:0,00:00:05,NotUserVisible,Information,System,NaN,GeneralMessage,uptime:The system has been running for 0 days ...
1,L:0,00:00:05,NotUserVisible,Information,System,NaN,GeneralMessage,ramfree:35 percent of memory in use!203366400 ...
2,L:0,00:00:05,NotUserVisible,Information,System,NaN,GeneralMessage,ssptasks:!Methods ...
3,L:0,00:00:04,UserVisible,Information,Lights,Scene,SceneChangedEvent,"['74', 'Night', 'Upstairs Landing', 'SceneReca..."
4,L:0,00:00:05,UserVisible,Information,System,Event,TimeClockChangedEvent,"['4', 'Upstairs Landing Off', 'Unassigned', 'E..."
5,L:0,00:00:05,UserVisible,Information,DoorLock,Device,SignalChangedEventWithStrings,"['208', 'Basement Mudroom Door', 'Basement Mud..."
6,L:0,00:00:05,UserVisible,Information,DoorLock,Scene,SceneChangedEvent,"['77', 'Lock Basement Mudroom Door', 'Basement..."
7,L:0,00:00:05,UserVisible,Information,System,Event,TimeClockChangedEvent,"['23', 'Lock Basement Door', 'Unassigned', 'Ev..."
8,L:0,00:00:05,NotUserVisible,Information,System,Cloud,GeneralMessage,Started nightlyFwCheckTimer with dueTime 1550000
9,L:0,00:00:06,NotUserVisible,Information,System,NaN,GeneralMessage,taskstat:App Name ...


In [11]:
df_dict = loaded_df.to_dict()

In [13]:
df_dict["eventType"][0]

'GeneralMessage'

# Testing part-by-part

In [22]:
import os 

for log in os.listdir('data'):
    os.rename('data/'+log, 'processed_data/'+log[-14::])

In [23]:
!ls processed_data

2017-04-21.bac	2017-05-04.bac	2017-05-17.bac	2017-05-30.bac	2017-06-12.bac
2017-04-22.bac	2017-05-05.bac	2017-05-18.bac	2017-05-31.bac	2017-06-13.bac
2017-04-23.bac	2017-05-06.bac	2017-05-19.bac	2017-06-01.bac	2017-06-14.bac
2017-04-24.bac	2017-05-07.bac	2017-05-20.bac	2017-06-02.bac	2017-06-15.bac
2017-04-25.bac	2017-05-08.bac	2017-05-21.bac	2017-06-03.bac	2017-06-16.bac
2017-04-26.bac	2017-05-09.bac	2017-05-22.bac	2017-06-04.bac	2017-06-17.bac
2017-04-27.bac	2017-05-10.bac	2017-05-23.bac	2017-06-05.bac	2017-06-18.bac
2017-04-28.bac	2017-05-11.bac	2017-05-24.bac	2017-06-06.bac	2017-06-19.bac
2017-04-29.bac	2017-05-12.bac	2017-05-25.bac	2017-06-07.bac	2017-06-20.bac
2017-04-30.bac	2017-05-13.bac	2017-05-26.bac	2017-06-08.bac	2017-06-21.bac
2017-05-01.bac	2017-05-14.bac	2017-05-27.bac	2017-06-09.bac
2017-05-02.bac	2017-05-15.bac	2017-05-28.bac	2017-06-10.bac
2017-05-03.bac	2017-05-16.bac	2017-05-29.bac	2017-06-11.bac


In [115]:
fname = 'processed_data/2017-04-27.bac'
lines = [line.rstrip('\n') for line in open(fname)]
info_lines = [[line[0:3],line[6:14]] +line[17:].split('|') for line in lines]
print(info_lines[3])

['L:0', '00:00:05', 'UserVisible', 'Information', 'DoorLock', 'Device', 'SignalChangedEventWithStrings', '208', 'Basement Mudroom Door', 'Basement Mudroom', '', 'Lock', 'ToDevice', '']


In [5]:
!mkdir processed_data